In [22]:
import pandas as pd
df = pd.read_csv("walmart-products.csv")

In [23]:
df.head()

,timestamp,url,final_price,sku,currency,gtin,specifications,image_urls,top_reviews,rating_stars,...,sizes,colors,seller,other_attributes,customer_reviews,ingredients,initial_price,discount,ingredients_full,categories
0,2024-08-24 00:00:00.000,https://www.walmart.com/ip/Laura-Mercier-Cavia...,22.90,173530386,USD,7.361501e+11,"[{""name"":""Brand"",""value"":""Laura Mercier""},{""na...","[""https://i5.walmartimages.com/seo/Laura-Merci...","{""negative"":{},""positive"":{}}","{""five_stars"":2,""four_stars"":3,""two_stars"":1}",...,[],"[""Sugar Frost"",""Tuxedo""]",Wal███t.c███,"[{""name"":""Instructions"",""value"":""Apply directl...","[{""name"":""Jac███"",""rating"":5,""review"":""My only...","Cyclopentasiloxane, trimethylsiloxysilicate, s...",NaN,NaN,"[{""type"":""Ingredients"",""values"":""Cyclopentasil...","[""Beauty"",""Makeup"",""Eye Makeup"",""Eye Shadow"",""..."
1,2024-08-24 00:00:00.000,https://www.walmart.com/ip/Exultantex-Grey-Bla...,47.88,430528189,USD,7.710779e+11,"[{""name"":""Brand"",""value"":""Exultantex""},{""name""...","[""https://i5.walmartimages.com/seo/Exultantex-...","{""negative"":{""rating"":1,""review"":""Color not ac...","{""five_stars"":47,""four_stars"":4,""one_star"":4,""...",...,"[""50\"" x 54\"""",""50\"" x 63\"""",""50\"" x 84\"""",""50...","[""Black"",""Blue"",""Green"",""Gray"",""Natural(Ivory)...",Exu███nte███ome███,"[{""name"":""Fabric Care Instructions"",""value"":""M...","[{""name"":""Dana"",""rating"":5,""review"":""I love th...",NaN,70.80,$22.92,NaN,"[""Home"",""Decor"",""Curtains & Window Treatments""..."
2,2024-08-24 00:00:00.000,https://www.walmart.com/ip/Jessica-London-Wome...,33.24,6013308220,USD,4.651920e+11,"[{""name"":""Features"",""value"":""Easy Care""},{""nam...","[""https://i5.walmartimages.com/seo/Jessica-Lon...","{""negative"":{},""positive"":{}}","{""five_stars"":2}",...,"[""S"",""M"",""L"",""1X"",""2X"",""3X""]","[""Aqua Sea"",""Dark Olive Green"",""Dark Sapphire""...",Ful███aut███ran███,"[{""name"":""Fabric Care Instructions"",""value"":""M...","[{""name"":""Bev███y F███"",""rating"":5,""review"":""V...",NaN,37.99,$4.75,NaN,"[""Clothing"",""Womens Plus"",""Plus Size Tops"",""Pl..."
3,2024-08-24 00:00:00.000,https://www.walmart.com/ip/100-Cotton-King-Per...,49.99,161657830,USD,8.407081e+11,"[{""name"":""Brand"",""value"":""Simply Put""},{""name""...","[""https://i5.walmartimages.com/asr/d88fe658-b9...","{""negative"":{""rating"":1,""review"":""[This review...","{""five_stars"":78,""four_stars"":17,""one_star"":2,...",...,"[""Queen"",""King""]","[""Beige"",""Blue"",""Gray"",""Spa Blue""]",IC ███bal███c,"[{""name"":""Fabric Care Instructions"",""value"":""M...","[{""name"":""Jes███oge███"",""rating"":5,""review"":""[...",NaN,71.99,$22.00,NaN,"[""Home"",""Bedding"",""Duvet Covers"",""King Duvet C..."
4,2024-08-25 00:00:00.000,https://www.walmart.com/ip/Disney-Boys-Graphic...,12.99,5397071399,USD,4.600086e+11,"[{""name"":""Country of Origin - Textiles"",""value...","[""https://i5.walmartimages.com/seo/Disney-Boys...","{""negative"":{""rating"":1,""review"":""This size sh...","{""five_stars"":12,""one_star"":1,""three_stars"":1,...",...,"[""2T"",""3T"",""3T-4T"",""3-4 Years"",""4-5 Years"",""4-...","[""1#Yellow Micky"",""2#Pink Minnie"",""3#Blue Dona...",Pat███,NaN,"[{""name"":""Mary"",""rating"":5,""review"":""Love thes...",NaN,29.00,$16.01,NaN,"[""Clothing"",""Kids Clothing"",""Boys Clothing"",""B..."


In [22]:
df['sku'].nunique()

1000

In [37]:
import cohere
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import pandas as pd
import json
from dotenv import load_dotenv
import os
from tqdm import tqdm

# === CONFIG ===
load_dotenv()
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
INDEX_NAME = "walmart-sparkathon-images"
EMBED_MODEL = "embed-v4.0"

# === INIT ===
co = cohere.ClientV2(COHERE_API_KEY)

pc = Pinecone(api_key=PINECONE_API_KEY)
pc.list_indexes()
index = pc.Index(INDEX_NAME)


# === LOAD DATA ===
df = pd.read_csv("walmart-products.csv")

# # === EMBED + UPSERT LOOP ===
# for _, row in tqdm(df.iterrows(), total=len(df)):
#     try:
#         specs = json.loads(row['specifications']) if pd.notna(row['specifications']) else []
#         spec_text = " | ".join([f"{s['name']}: {s['value']}" for s in specs])

#         review_data = json.loads(row['customer_reviews']) if pd.notna(row['customer_reviews']) else []
#         review_text = " | ".join([f"{r.get('name', '')}: {r.get('review', '')}" for r in review_data])

#         other_attrs = row['other_attributes'] if pd.notna(row['other_attributes']) else ""
#         ingredients = row['ingredients'] if pd.notna(row['ingredients']) else ""
#         sizes = row['sizes'] if pd.notna(row['sizes']) else ""
#         colors = row['colors'] if pd.notna(row['colors']) else ""
#         top_reviews = row['top_reviews'] if pd.notna(row['top_reviews']) else ""
#         rating = row['rating'] if pd.notna(row['rating']) else ""
#         main_image = row['main_image'] if pd.notna(row['main_image']) else ""
#         description = row['description'] if pd.notna(row['description']) else ""
#         category_name = row['category_name'] if pd.notna(row['category_name']) else ""
#         currency = row['currency'] if pd.notna(row['currency']) else ""

#         text = f"""
#         Product SKU: {row['sku']}
#         Title/Description: {description}
#         Brand Info: {spec_text}
#         Categories: {row['categories']}
#         Category Name: {category_name}
#         Price: {row['final_price']} {currency}
#         Currency: {currency}
#         Top Reviews: {top_reviews}
#         Ratings (out of 5): {rating}
#         Sizes: {sizes}
#         Colours: {colors}
#         Main Image: {main_image}
#         Other Attributes: {other_attrs}
#         Customer Reviews: {review_text}
#         Ingredients: {ingredients}
#         """

#         # === Get Embedding from Cohere ===
#         response = co.v2.embed(
#             texts=[text],
#             model=EMBED_MODEL,
#             output_dimension=1024,
#             input_type="search_document",  # use "search_query" when embedding queries
#             embedding_types=["float"]  # specify embedding type
#         )
#         vector = response.embeddings.float[0]



#         # === Upsert to Pinecone ===
#         index.upsert([{
#             "id": str(row["sku"]),
#             "values": vector,
#             "metadata": {
#                 "url": row["url"],
#                 "price": row["final_price"],
#                 "brand": [s["value"] for s in specs if s["name"].lower() == "brand"][0] if specs else None,
#                 "categories": row["categories"]
#             }
#         }])
#         break



#     except Exception as e:
#         print(f"Failed on row {row['sku']}: {e}")
#         break

In [ ]:
response = co.v2.embed(
            texts=["100% ORGANIC COTTON Queen size bedsheet pink color"],
            model=EMBED_MODEL,
            output_dimension=1024,
            input_type="search_document",
            embedding_types=["float"]  # specify embedding type
        )
vector = response.embeddings.float[0]

In [42]:
import pyperclip 
pyperclip.copy(vector)

In [ ]:
import cohere
from PIL import Image
from io import BytesIO
import base64
import requests

df = pd.read_csv("walmart-products.csv")

def url_to_base64(image_url):
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))
    image_format = img.format.lower()
    buffered = BytesIO()
    img.save(buffered, format=img.format)
    img_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return f"data:image/{image_format};base64,{img_base64}"


# for _, row in tqdm(df.iterrows(), total=len(df)):
#     try:
#         specs = json.loads(row['specifications']) if pd.notna(row['specifications']) else []
#         spec_text = " | ".join([f"{s['name']}: {s['value']}" for s in specs])

#         review_data = json.loads(row['customer_reviews']) if pd.notna(row['customer_reviews']) else []
#         review_text = " | ".join([f"{r.get('name', '')}: {r.get('review', '')}" for r in review_data])
#         other_attrs = row['other_attributes'] if pd.notna(row['other_attributes']) else ""
#         sizes = row['sizes'] if pd.notna(row['sizes']) else ""
#         main_image = row['main_image'] if pd.notna(row['main_image']) else ""

        
#         input = {
#             "content": [
#             {"type": "image_url", "image_url": {"url": url_to_base64(main_image[1:-1])}}
#             ]
#         }

#         # === Get Embedding from Cohere ===
#         response = co.v2.embed(
#             inputs=[input],
#             model=EMBED_MODEL,
#             output_dimension=1024,
#             input_type="search_document",
#             embedding_types=["float"]  # specify embedding type
#         )
#         image_vector = response.embeddings.float[0]


#         # === Upsert to Pinecone ===
#         index.upsert([{
#             "id": str(row["sku"]),
#             "values": image_vector,
#             "metadata": {
#                 "url": row["url"],
#                 "price": row["final_price"],
#                 "brand": [s["value"] for s in specs if s["name"].lower() == "brand"][0] if specs else None,
#                 "categories": row["categories"]
#             }
#         }])



#     except Exception as e:
#         print(f"Failed on row {row['sku']}: {e}")


  0%|          | 0/1000 [00:00<?, ?it/s]

['https://i5.walmartimages.com/seo/Laura-Mercier-Caviar-Stick-Eye-Color-Sugar-Frost-1-64g-0-05oz_55297223-7af5-4a30-83c2-4d74d08969e3.8ca12dd7578ff564b6e01923e85ffd11.jpeg']


  0%|          | 1/1000 [00:16<4:33:55, 16.45s/it]

['https://i5.walmartimages.com/seo/Exultantex-Grey-Blackout-Curtains-for-Living-Room-Pom-Pom-Thermal-Window-Curtains-50-W-x-95-L-2-Panels-Rod-Pocket_f89aea0d-9d30-4bce-b65e-8c15732f1ca4.b20ae059333c322a0c8ea79bd569c7c7.jpeg']


  0%|          | 2/1000 [00:20<2:32:41,  9.18s/it]

...

['https://i5.walmartimages.com/seo/Cocokind-Vitamin-C-Serum-Glow-Facial-Serum-Hydrating-Brightening-with-Hyaluronic-Acid-1-Oz_2e37617e-2380-4de4-a069-907308fea37d.893c85e8d167696b29a44f012f4966c5.jpeg']


 10%|█         | 101/1000 [05:35<56:34,  3.78s/it]

['https://i5.walmartimages.com/seo/NINA-NEW-YORK-Womens-Silver-Goring-Embellished-Nadette-Round-Toe-Wedge-Slip-On-Dress-Sandals-Shoes-5-5-M_6fbca8ed-90fc-41ea-a9dd-a7a223f2d2ba.f566d185c91457f7b67ea0a26be0ed86.jpeg']


 10%|█         | 101/1000 [05:37<50:02,  3.34s/it]


KeyboardInterrupt: 

In [16]:
import pyperclip 
pyperclip.copy(vector)

In [14]:
response = co.v2.embed(
        texts=["एक मेकअप आइटम जो मेरी त्वचा की दीर्घायु को बढ़ाएगा"],
        model=EMBED_MODEL,
        output_dimension=1024,
        input_type="search_document",  # use "search_query" when embedding queries
        embedding_types=["float"]  # specify embedding type
    )
vector = response.embeddings.float[0]


In [18]:
results=index.query(
    vector=vector,
    top_k=5,
    include_values=True,
    include_metadata=True
)

In [59]:
pc.list_indexes()

[
    {
        "name": "sparkathon-walmart",
        "metric": "cosine",
        "host": "sparkathon-walmart-qt6qhli.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 1024,
        "deletion_protection": "enabled",
        "tags": null
    }
]

In [19]:
for match in results["matches"]:
    print("ID:", match["id"])
    print("Score:", match["score"])
    print("URL:", match["metadata"]["url"])
    print("-" * 40)

ID: 3440558193
Score: 0.2975403
URL: https://www.walmart.com/ip/Cocokind-Vitamin-C-Serum-Glow-Facial-Serum-Hydrating-Brightening-with-Hyaluronic-Acid-1-Oz/3440558193
----------------------------------------
ID: 908232424
Score: 0.27275896
URL: https://www.walmart.com/ip/GrandeLash-MD-Serum-Lash-Enhancing-0-07-oz-3-Month-Suply/908232424
----------------------------------------
ID: 5062553620
Score: 0.22299968
URL: https://www.walmart.com/ip/SmartMouth-Original-Activated-Breath-Rinse-16-oz-Adult/5062553620
----------------------------------------
ID: 102593173
Score: 0.20512384
URL: https://www.walmart.com/ip/Hanes-Beefy-T-Unisex-Long-Sleeve-T-Shirt-2-Pack-Deep-Royal-S/102593173
----------------------------------------
ID: 179112912
Score: 0.19541287
URL: https://www.walmart.com/ip/Solaray-Food-Carotene-Vitamin-A-as-Beta-Carotene-25000IU-Carotenoids-for-Healthy-Skin-Eyes-Antioxidant-Activity-Immune-System-Support-200ct/179112912
----------------------------------------


In [29]:
print(type(results["matches"][0]["id"]))

<class 'str'>


In [39]:
from PIL import Image
from io import BytesIO
import base64
import requests

def url_to_base64(image_url):
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))
    image_format = img.format.lower()
    buffered = BytesIO()
    img.save(buffered, format=img.format)
    img_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return f"data:image/{image_format};base64,{img_base64}"
image_url = url_to_base64("https://i5.walmartimages.com/asr/615e6ce3-68cd-466b-b53f-323792898479.227b7ca15cbaa67e91d1053ef00d5210.jpeg")

In [41]:
response = co.v2.embed(
            images=[image_url],
            model=EMBED_MODEL,
            output_dimension=1024,
            input_type="search_document",
            embedding_types=["float"]  # specify embedding type
        )

In [46]:
response.embeddings.float[0]

[0.027709961,
 -0.028442383,
 0.040527344,
 0.009216309,
 -0.046875,
 -0.0065612793,
 -0.022949219,
 0.041992188,
 -0.015625,
 -0.031982422,
 0.025756836,
 0.020507812,
 0.013977051,
 -0.045410156,
 0.046142578,
 -0.018432617,
 -0.008422852,
 0.024291992,
 0.016967773,
 0.00340271,
 -0.010070801,
 0.03125,
 0.0064697266,
 -0.03515625,
 0.052490234,
 -0.015625,
 0.041503906,
 0.03112793,
 -0.040527344,
 -0.03881836,
 -0.00680542,
 -0.013366699,
 0.0234375,
 -0.026123047,
 -0.042236328,
 -0.017822266,
 0.0064697266,
 -0.022216797,
 0.023071289,
 -0.037353516,
 0.029418945,
 -0.025390625,
 -0.018676758,
 -0.02758789,
 0.017333984,
 -0.020507812,
 -0.017578125,
 -0.007873535,
 -0.010070801,
 -0.041748047,
 0.048583984,
 0.0077209473,
 -0.011779785,
 0.064941406,
 0.02709961,
 0.029907227,
 0.03955078,
 -0.014709473,
 -0.03173828,
 -0.02709961,
 -0.003112793,
 -0.087402344,
 0.03173828,
 -0.028564453,
 0.028930664,
 -0.020507812,
 0.0099487305,
 -0.04272461,
 -0.068847656,
 -0.026611328,
 -